In [2]:
import os
# 下面注释掉的适合在非Pycharm 的Juypter Notebook上使用
# import sys
# path = os.path.join(os.path.dirname(os.getcwd()))
# sys.path.append(path) # 将所需要的根目录添加到路径
import torch
import numpy as np
import pandas as pd
# 导入模型可视化和训练过程可视化库
from IPython.display import clear_output as clear
# 导入模型
from models.supervisedModels.CNNRNNs import Conv2DRNNs
# 导入小工具
from utils.common_utils import printlog
from trainTest.datasets.dataset_utils import get_fileName_weights, get_save_path, get_intra_dataloaders
from trainTest.train.train_dl_models import train_test_model

In [3]:
### 1. 获取文件路径和文件名的设置
file_path = os.path.join(os.path.dirname(os.getcwd()) , 'preProcessing', 'trainData')
subjects_list_global = list(['AB156','AB185','AB186','AB188','AB189','AB190','AB191','AB192','AB193','AB194'])

### 2. 获取dataloaders的设置
total_exp_time = 5
train_batch, valid_batch, test_batch = 32, 32, 32
# modals = ['E', 'A', 'E-A']
### 3. 模型训练和测试的参数设置
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
## 4.1. 迭代次数、初始学习率、优化器、学习率衰减、早停和损失函数的设置
# 1)'epoch': int, 最大训练轮数
# 2)'initial_lr': 初始学习率，默认0.01
# 3)'optimizer': 优化器，默认'Adam'，one of ['Adam', 'RMSprop']
# 4)'lr_scheduler': 学习率衰减，scheduler_type: one of ['None', 'StepLR', 'MultiStepLR', 'ExponentialLR', AutoWarmupLR', 
#                                                   'GradualWarmupLR' ,'ReduceLROnPlateau']
# 5)'early_stopping': 早停
# 6)'criterion': 'loss_type', 默认'CE'('CrossEntropy')，one of ['CE', 'WeightedCE', 'FL'('FocalLoss'), 'WeightedFL', 'AttenuationWeightedCE', 'AttenuationWeightedFL']
#                'modify_type': 'exponent' / 'linear'
max_epoch = 100
callbacks = {'epoch': max_epoch,
             'initial_lr': 0.01*np.sqrt(0.1),
             'optimizer': 'Adam',
             'lr_scheduler': {'scheduler_type': 'ReduceLROnPlateau',
                              'params':{
                                  'StepLR':{'step_size': int(0.2*max_epoch), 'gamma': np.sqrt(0.1)},
                                  'MultiStepLR':{'milestones': [int(0.2*max_epoch), int(0.4*max_epoch), int(0.6*max_epoch), int(0.8*max_epoch)], 
                                                 'gamma': np.sqrt(0.1)},
                                  'ExponentialLR':{'gamma': 0.9},
                                  'AutoWarmupLR':{'num_warm': 10},
                                  'GradualWarmupLR':{'multiplier': 1, 'total_epoch': 10},
                                  'ReduceLROnPlateau':{'mode': 'max', 'factor': np.sqrt(0.1),
                                                       'patience': 9, 'verbose': False,
                                                       'threshold': 0.00001, 'min_lr': 0.0001},
                              }
                              },
             'early_stopping': {'use_es': True, 'params':{'patience': 15, 'verbose': False, 'delta': 0.00001}},
             'criterion': {'loss_type': 'CE', 'params':{'modify_type': 'exponent', 'exponent_factor': 5}}}

## 4.2 训练和测试过程中的画图和保存设置
# 1) use_tqdm: 为True使用进度条在进度条内打印输出，为False使用hiddenlayer的History打印输出
# 2) train_plot: 是否使用hiddenlayer的Canvas为训练过程画图
# 3) print_interval: 多少个epoch打印一次输出或更新一次tqdm
# 4) model_eval: 使用验证集和测试集评估模型前是否打开model.eval()
# 5) test_metrics: list, 整个训练结束后在测试集上评估的指标 ['accuracy', 'precision', 'recall', 'f1', 'specificity', 'npv']
# 6) confusion_matrix: dist, 参考'metrics/get_test_metrics函数'. cmap: 'YlGnBu' / 'Blues'
#                     'show_type':  {'cm', 'normalized_cm', 'all'}
# 7) tsne_visualization: dist, 参考'visualization/tsne函数'
#    'show_type':  'train_set' / 'test_set' / 'all', 'feature_type': 'CNN' / 'Linear'
train_test_utils = {'use_tqdm': False,
                    'train_plot': False,
                    'print_interval': 1,
                    'model_eval': {'valid': True, 'test': True},
                    'test_metrics': ['accuracy', 'precision', 'recall', 'f1', 'specificity', 'npv'],
                    'confusion_matrix': {'get_cm': True, 'params':{'show_type': 'all', 'plot': True, 'save_fig': True,
                                                                   'save_results': True, 'cmap': 'YlGnBu'}},
                    'tsne_visualization': {'get_tsne': False, 'params': {'show_type': 'all', 'feature_type': 'CNN', 
                                                                        'save_results': True, 'save_fig': True}},
                    }    

In [ ]:
total_exp_time = 5
settings_dict = {'total_exp_time': total_exp_time}
criterion = 'CE'
callbacks['criterion']['loss_type'] = criterion
train_test_utils['confusion_matrix']['get_cm'] = True
train_test_utils['tsne_visualization']['get_tsne'] = False

# 要更改的参数
modals = ['E', 'A', 'E-A']
attention_types = ['MC-ATT', 'No-ATT']

for modal in modals:
    for attention_type in attention_types:
        model = Conv2DRNNs(modal, attention_type=attention_type)
        model_name = model.get_model_name()
        
        ## 开始训练和测试
        for subject_order in range(len(subjects_list_global)):
            subject = subjects_list_global[subject_order]
            file_name, class_weights, encoded_label_name, raw_label_type = get_fileName_weights(file_path,subject, subjects_list_global)
            basic_save_path = os.path.join(os.path.dirname(os.getcwd()) , 'results', 'CNN-RNNs', modal)
            save_path = get_save_path(basic_save_path, model_name, subject)
            print('保存结果的绝对路径： ', save_path['absolute_path'])
            print('保存结果的相对路径： ', save_path['relative_path'])
            train_test_utils['confusion_matrix']['params']['label_type'] = raw_label_type
            callbacks['weights'] = class_weights
            
            for exp_tim in range(total_exp_time):
                clear()
                current_exp_time = exp_tim + 1
                settings_dict['current_exp_time'] = current_exp_time
                printlog(info='当前模态：%s, 当前模型：%s'% (modal, model_name), time=True, line_break=False)
                printlog(info='当前受试者编号：%s' % subject, time=True, line_break=False)
                printlog(info='当前实验次数：%d / %d' % (current_exp_time, total_exp_time), time=True, line_break=False)
                train_loader, valid_loader, test_loader = get_intra_dataloaders(file_name, encoded_label_name, total_exp_time, current_exp_time, train_batch, test_batch, valid_batch, modal)
                print('Sample size of train set, valid set and test set are: ', len(train_loader.dataset), len(valid_loader.dataset), len(test_loader.dataset))
                model = Conv2DRNNs(modal, attention_type=attention_type)
                model.double()
                model.to(device=device)
                train_test_model(settings_dict, model, train_loader, valid_loader, test_loader, device, save_path, callbacks, train_test_utils)
                
        subjects_list_global = list(['AB156','AB185','AB186','AB188','AB189','AB190','AB191','AB192','AB193','AB194'])
        # 当一个模型针对所有受试者全部训练测试后， 计算保存所有受试者的平均结果
        # 新增一个记录所有受试者的所有测试结果的df1
        df1_metrics = []
        df2_metrics_mean = []
        df2_metrics_std = []
        printlog(info='当前模型：%s' % model_name, time=True, line_break=True)
        basic_file_path = os.path.join(os.path.dirname(os.getcwd()),  'results', 'CNN-RNNs', modal)

        for subject_order in range(len(subjects_list_global)):
            subject = subjects_list_global[subject_order]
            metrics_file_path = get_save_path(basic_file_path, model_name, subject)
            metrics_file_name = os.path.join(metrics_file_path['absolute_path'], 'test_metrics.csv')
            # 判断文件是否存在
            if not os.path.exists(metrics_file_name):
                print("受试者：%s 的文件: %s， 不存在！" %(subject, metrics_file_name))
            else:
                # 读取每个受试者的test_metrics
                print("读取受试者：%s 的test_metrics: " %subject)
                df = pd.read_csv(metrics_file_name, header=0, index_col=0)
                # ignore_index=True参数用于重置索引，以确保索引是连续的
                df1_metrics.extend(df.T.values[:-2, :])
                df2_metrics_mean.append(df.T.values[-2, :])
                df2_metrics_std.append(df.T.values[-1, :])
    
        printlog(info='当前模型：%s, 保存所有受试者所有测试指标的平均结果' % model_name, time=False, line_break=False)
        df1 = pd.DataFrame(df1_metrics, index=range(1, len(df1_metrics) + 1), columns=df.index)
        # 计算平均值并添加到DataFrame
        mean_row = df1.mean().to_frame().T  # 转换为DataFrame并进行转置
        mean_row.index = ['mean']  # 设置索引名称
        df1 = pd.concat([df1, mean_row])
        # 计算标准差并添加到DataFrame
        std_row = df1[:-1].std().to_frame().T  # 转换为DataFrame并进行转置，排除最后一行(mean行)来计算标准差
        std_row.index = ['std']  # 设置索引名称
        df1 = pd.concat([df1, std_row]).round(3)
        # 保存df1  
        dfs_save_path = os.path.dirname(metrics_file_path['absolute_path'])
        df1_save_name = os.path.join(dfs_save_path, 'all_metrics_averaged_results.csv')
        df1.to_csv(df1_save_name, index=True)

        printlog(info='当前模型：%s, 保存单个受试者测试指标平均的平均结果' % model_name, time=False, line_break=False)
        # 保存df2
        df2_metrics_mean, df2_metrics_std = np.round(np.array(df2_metrics_mean), 3), np.round(np.array(df2_metrics_std), 3)
        df2_metrics = np.array([str(df2_metrics_mean[i, j]) +'+'+ str(df2_metrics_std[i, j]) for i in range(df2_metrics_mean.shape[0]) for j in range(df2_metrics_mean.shape[1])])
        df2_metrics = df2_metrics.reshape(df2_metrics_mean.shape)
        df2 = pd.DataFrame(df2_metrics, index=['Sub'+i for i in subjects_list_global], columns=df.index)
        # 计算平均值并添加到DataFrame
        mean_row = np.round(np.mean(df2_metrics_mean, axis=0), 3)
        std_row = np.round(np.std(df2_metrics_mean, axis=0), 3)
        df2.loc['mean'] = mean_row
        df2.loc['std'] = std_row
        df2_save_name = os.path.join(dfs_save_path, 'alone_subject_averaged_results.csv')
        df2.to_csv(df2_save_name, index=True)